In [10]:
#%matplotlib notebook
import sys
import os
import glob

# Standard modules used through the notebook 
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

from bundle_adjust import data_loader as loader
from bundle_adjust import ba_timeseries
import pickle

# Display and interface settings (just for the notebook interface)
%matplotlib inline
%load_ext autoreload
%autoreload 2
np.set_printoptions(linewidth=150)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Check std single date (in the overlapping areas) and std along time

In [3]:
output_dir = 'exp/mine_L1A'
ba_method = 'ba_global'

# get median std per date
std_per_date_fnames = glob.glob(os.path.join(output_dir, '{}/4D/4Dstats/std_per_date/*.tif'.format(ba_method)))
stacked_std_per_date = np.dstack([np.array(Image.open(fn)) for fn in std_per_date_fnames])
n_dates = stacked_std_per_date.shape[2]
median_std_per_date = [np.nanmedian(stacked_std_per_date[:,:,i], axis=(0, 1)) for i in range(n_dates)]
                                 
# get std along time
std_along_time_fname = os.path.join(output_dir, '{}/4D/4Dstats/std_along_time.tif'.format(ba_method))
std_along_time = np.array(Image.open(std_along_time_fname))

print('bundle adjustment method:', ba_method)

print('average median std per date: {:.2f}'.format(np.mean(median_std_per_date)))

print(median_std_per_date)

print('median std along time: {:.2f}'.format(np.nanmedian(std_along_time, axis=(0, 1))))

ValueError: need at least one array to concatenate

### Check reprojection error

In [4]:
scene = ba_timeseries.Scene('morenci_L1A_config.json')
scene.display_aoi()

#############################################################

Loading scene from morenci_L1A_config.json

-------------------------------------------------------------
Configuration:
    - images_dir:      /home/anger/for_roger/L1A_Fast-rpcfix
    - s2p_configs_dir: /home/anger/for_roger/s2p
    - rpc_src:         geotiff
    - output_dir:      exp/mine_L1A
-------------------------------------------------------------

Defining aoi from s2p config.json files... 233 / 233 done

Number of acquisition dates: 1 (from 2019-01-28 to 2019-01-28)
Number of images: 101
The aoi covers a total of 61.63 squared km:

#############################################################




Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [5]:
# load scene and timeline indices
timeline_indices = [0]

scene.get_timeline_attributes(timeline_indices, ['datetime', 'n_images'])

index  |  datetime             |  n_images
__________________________________________

0      |  2019-01-28 20:47:10  |  101     
__________________________________________

                                  101 total




In [16]:
def eval_reprojection_error_from_rpcs(scene, myimages, ba_method, C, pairs_to_triangulate):
    
    
    # load filenames and rpcs
    rpcs_init_dir = '{}/RPC_init'.format(scene.dst_dir)
    rpcs_init = loader.load_rpcs_from_dir(myimages, rpcs_init_dir, suffix='RPC', verbose=True)
    rpcs_adj_dir = '{}/{}/RPC_adj'.format(scene.dst_dir, ba_method)
    rpcs_adj = loader.load_rpcs_from_dir(myimages, rpcs_adj_dir, suffix='RPC_adj', verbose=True)
    
    # load image crops
    mycrops = loader.load_image_crops(myimages, verbose=True)
    
    # init ba
    cam_model = 'Perspective'
    ba_input_data = {}
    ba_input_data['input_dir'] = scene.dst_dir
    ba_input_data['output_dir'] = scene.dst_dir
    ba_input_data['n_new'] = len(myimages)
    ba_input_data['n_adj'] = 0
    ba_input_data['image_fnames'] = myimages
    ba_input_data['crops'] = mycrops
    ba_input_data['masks'] = None
    ba_input_data['rpcs'] = rpcs_init
    ba_input_data['cam_model'] = cam_model
    ba_input_data['aoi'] = scene.aoi_lonlat
    from bundle_adjust.ba_pipeline import BundleAdjustmentPipeline
    ba_pipeline = BundleAdjustmentPipeline(ba_input_data)
    
    # approximate rpcs as proj matrices
    from bundle_adjust.ba_core import approximate_rpcs_as_proj_matrices
    P_crop_ba = approximate_rpcs_as_proj_matrices(rpcs_adj, mycrops, scene.aoi_lonlat, cam_model)
    ba_pipeline.P_crop_ba = P_crop_ba
        
    # init pts 3D
    from bundle_adjust.ba_triangulation import initialize_3d_points
    pts_3d = initialize_3d_points(ba_pipeline.input_P, C, pairs_to_triangulate, cam_model)
    pts_3d_ba = initialize_3d_points(P_crop_ba, C, pairs_to_triangulate, cam_model)
        
    # compute reprojection error
    ba_pipeline.pts_3d = pts_3d
    ba_pipeline.pts_3d_ba = pts_3d_ba
    ba_pipeline.C = C
    avg_init, avg_ba = [], []
    for im_idx, fn in enumerate(myimages):
        e_init, e_ba, _, _, _ = ba_pipeline.compute_reproj_err_per_image(im_idx)
        avg_init.append(np.mean(e_init))
        avg_ba.append(np.mean(e_ba))
        
        #ba_pipeline.analyse_reproj_err_particular_image(im_idx, plot=True)
        
        print('{:02}: avg init err {:.2f}, avg ba err {:.2f}       {}'.format(im_idx, 
                                                                              avg_init[-1],
                                                                              avg_ba[-1],
                                                                              os.path.basename(fn)))
        
    
    print('\nMethod: {} bundle adjustment'.format(ba_method))
    print('-----------------------------------------------------------------')
    print('\navg init err per image: {:.2f}'.format(np.mean(avg_init)))
    print('\navg ba err per image: {:.2f}'.format(np.mean(avg_ba)))

In [12]:
ba_method = 'ba_global' #'out-of-core'

pickle_in = open('{}/{}/matches.pickle'.format(scene.dst_dir, ba_method),'rb')
pairwise_matches = pickle.load(pickle_in)
pickle_in = open('{}/{}/pairs_triangulation.pickle'.format(scene.dst_dir, ba_method),'rb')
pairs_to_triangulate = pickle.load(pickle_in)
pickle_in = open('{}/{}/filenames.pickle'.format(scene.dst_dir, ba_method),'rb')
myimages = pickle.load(pickle_in)


#forbbiden_pairs = [(1,12), (12,16)]
#if len(forbbiden_pairs) >0:
#    pairs_to_triangulate = list(set(pairs_to_triangulate) - set(forbbiden_pairs))

#allowed_im_indices = [14, 15, 16, 17, 18, 19, 20]
#allowed_pairs = [(im_i, im_j) for im_i in allowed_im_indices for im_j in allowed_im_indices if im_i != im_j and im_i<im_j]

#allowed_pairs = [(1,12), (12,16)]
#pairs_to_triangulate = list(set(pairs_to_triangulate).intersection(set(allowed_pairs)))


features = []
for fn in myimages:
    f_id = os.path.splitext(os.path.basename(fn))[0]
    features.append(pickle.load(open('{}/{}/features/{}.pickle'.format(scene.dst_dir, ba_method, f_id),'rb')))
    
from feature_tracks.ft_utils import feature_tracks_from_pairwise_matches
C, _ = feature_tracks_from_pairwise_matches(features, pairwise_matches, pairs_to_triangulate)

print('Using {} feature tracks\n'.format(C.shape))

print('pairs to triangulate:', pairs_to_triangulate)

C.shape before baseline check (202, 29577)
C.shape after baseline check (202, 20678)
Using (202, 20678) feature tracks

pairs to triangulate: [(4, 41), (3, 35), (3, 97), (28, 41), (44, 70), (69, 77), (0, 76), (29, 44), (52, 73), (42, 97), (38, 86), (46, 78), (37, 97), (57, 86), (20, 48), (28, 88), (29, 70), (34, 80), (65, 89), (52, 98), (64, 88), (20, 75), (55, 78), (6, 98), (36, 89), (12, 94), (37, 85), (36, 84), (51, 73), (44, 97), (49, 73), (60, 81), (13, 35), (0, 45), (14, 77), (62, 75), (11, 78), (34, 79), (59, 72), (5, 34), (18, 65), (41, 90), (48, 80), (15, 52), (38, 92), (12, 39), (63, 83), (67, 96), (50, 90), (53, 73), (61, 97), (1, 38), (49, 89), (6, 86), (19, 96), (40, 92), (26, 38), (47, 90), (29, 59), (27, 87), (42, 94), (19, 70), (9, 34), (8, 54), (35, 96), (58, 93), (39, 88), (51, 78), (6, 57), (23, 65), (4, 84), (11, 57), (34, 91), (14, 75), (3, 96), (16, 72), (22, 69), (53, 99), (18, 71), (2, 64), (69, 89), (7, 37), (16, 32), (10, 58), (68, 88), (11, 51), (60, 88), (17

In [17]:
eval_reprojection_error_from_rpcs(scene, myimages, ba_method, C, pairs_to_triangulate)

Loading 101 image rpcs / 101

Loading 101 image rpcs / 101

Loading 101 image crops / 101

Approximating RPCs as Perspective projection matrices
101 projection matrices / 101 (0 err)
Done!

Approximating RPCs as Perspective projection matrices
101 projection matrices / 101 (0 err)
Done!

00: avg init err 0.44, avg ba err 0.31       20190128_204710_ssc6d1_0014_basic_l1a_panchromatic_dn.tif
01: avg init err 0.56, avg ba err 0.33       20190128_204710_ssc6d3_0005_basic_l1a_panchromatic_dn.tif
02: avg init err 0.47, avg ba err 0.33       20190128_204710_ssc6d1_0010_basic_l1a_panchromatic_dn.tif
03: avg init err 1.39, avg ba err 1.33       20190128_204710_ssc6d2_0017_basic_l1a_panchromatic_dn.tif
04: avg init err 0.63, avg ba err 0.34       20190128_204710_ssc6d2_0013_basic_l1a_panchromatic_dn.tif
05: avg init err 0.39, avg ba err 0.28       20190128_204710_ssc6d1_0004_basic_l1a_panchromatic_dn.tif
06: avg init err 0.51, avg ba err 0.34       20190128_204710_ssc6d3_0006_basic_l1a_panchromat

In [408]:
P1 = np.array([[ 4.85128086e-02, -8.83537890e-02,  1.40268335e-03,  3.66886552e+04],
               [ 3.97942353e-02,  2.32994575e-02,  8.98703082e-02,  1.70862603e+04],
               [-1.08059602e-07, -5.74392857e-08,  7.96691137e-08,  1.00000000e+00]])

P2 =  np.array([[ 4.85869516e-02, -8.82992204e-02,  1.35108588e-03,  3.60184995e+04],
                [ 3.95414376e-02,  2.31577511e-02,  9.00036459e-02,  1.90280930e+04],
                [-1.08160881e-07, -5.78021362e-08,  7.91655350e-08,  1.00000000e+00]])

from bundle_adjust import ba_utils

Pr = ba_utils.relative_extrinsic_matrix_between_two_proj_matrices(P2, P1, verbose=True)

np.allclose(P2, P1 @ Pr)

[R1 | t1] = [R2 | t2] @ [R21 | t21] ? True
Found a rotation of 0.24595336226186001 degrees between both cameras



False

In [410]:
from bundle_adjust import ba_core

k1, r1, t1, o1 = ba_core.decompose_perspective_camera(P1)

ext1 = np.hstack(( r1, t1[:, np.newaxis] ))

In [411]:
np.allclose( P1, k1 @ ext1)

True

In [412]:
np.allclose(P2, k2 @ ext1 @ Pr)

True